<a href="https://colab.research.google.com/github/squeeko/DL_TF20_KerasCNNGANSRNNNLP/blob/in_progress/DL_TF2_Ch2_TF_Estimators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Estimators

TensorFlow provides Estimators as higher-level APIs, to provide scalable and
production-oriented solutions. They take care of all behind-the-scene activities such
as creating computational graphs, initializing the variables, training the model,
saving checkpoints, and logging TensorBoard files. TensorFlow provides two types
of Estimators:

• Canned Estimators: These are premade Estimators available in the
TensorFlow estimator module. These are models in a box; you just pass
them the input features and they are ready to use. Some examples are
Linear Classifier, Linear Regressor, DNN Classifier, and so on.

• Custom Estimators: Users can also create their own estimators from the
models they build in TensorFlow Keras. These are user-defined Estimators.
Before being able to use TensorFlow Estimator let us understand two important
components of the Estimator pipeline:

## Feature columns

The feature_column module of TensorFlow 2.0 acts as a bridge between your input
data and the model. The input parameters to be used by the estimators for training
are passed as feature columns. They are defined in TensorFlow feature_column and
specify how the data is interpreted by the model. To create feature columns we will
need to call functions from tensorflow.feature_columns. There are nine functions
available in feature column:

**• categorical_column_with_identity:** Here each category is one-hot
encoded, and thus has a unique identity. This can be used for numeric values
only.

**• categorical_column_with_vocabulary_file:** This is used when the
categorical input is a string and the categories are given in a file. The string is
first converted to a numeric value and then one-hot encoded.

**• categorical_column_with_vocabulary_list:** This is used when the categorical input is a string and the categories are explicitly defined in a list.
The string is first converted to a numeric value and then one-hot encoded.

**• categorical_column_with_hash_bucket:** In case the number of categories is very large, and it is not possible to one-hot encode, we use hashing.

**• crossed_column:** When we want to use two columns combined as one
feature, for example, in the case of geolocation-based data it makes sense to
combine longitude and latitude values as one feature.

**• numeric_column:** Used when the feature is a numeric, it can be a single value or even a matrix.

**• indicator_column:** We do not use this directly. Instead, it is used with the categorical column, but only when the number of categories is limited and can be represented as one-hot encoded.

**• embedding_column:** We do not use this directly. Instead, it is used with the categorical column, but only when the number of categories is very large and
cannot be represented as one-hot encoded.

**• bucketized_column:** This is used when, instead of a specific numeric value, we split the data into different categories depending upon its value.

The first six functions inherit from the Categorical Column class, the next three inherit from the Dense Column class, and the last one inherits from both classes. 

In the following example we will use numeric_column and categorical_column_ with_vocabulary_list functions.